In [3]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS  
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
company_file = "Company Data.docx"
rfp_file = "ELIGIBLE RFP - 1.pdf"

company_loader = Docx2txtLoader(company_file)
rfp_loader = PyPDFLoader(rfp_file)

company_docs = company_loader.load()
rfp_docs = rfp_loader.load()


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)


company_chunks = splitter.split_documents(company_docs)
rfp_chunks = splitter.split_documents(rfp_docs)

embeddings = OpenAIEmbeddings()


company_vectorstore = FAISS.from_documents(company_chunks, embeddings)
rfp_vectorstore = FAISS.from_documents(rfp_chunks, embeddings)

company_vectorstore.save_local("company_index")
rfp_vectorstore.save_local("rfp_index")
print("Company and RFP vectorstores have been created and saved locally.")

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 145 0 (offset 0)


In [4]:
from langchain_community.vectorstores import FAISS

In [6]:
company_vectorstore = FAISS.load_local("company_index", embeddings,allow_dangerous_deserialization=True)
rfp_vectorstore = FAISS.load_local("rfp_index", embeddings,allow_dangerous_deserialization=True)

company_relevant = company_vectorstore.similarity_search("Company certifications and qualifications", k=3)
rfp_relevant = rfp_vectorstore.similarity_search("Eligibility criteria for proposal submission", k=3)

company_chunk = "\n\n".join([doc.page_content for doc in company_relevant])
rfp_chunk = "\n\n".join([doc.page_content for doc in rfp_relevant])

In [26]:
from langchain_core.prompts import PromptTemplate
from langchain.agents import Agent
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain




In [54]:
 

eligibility_prompt = PromptTemplate.from_template("""**Role:** You are an automated RFP Compliance Analyst.

**Objective:** Determine if the company described in the 'COMPANY PROFILE' section is legally eligible to bid based *only* on the mandatory requirements listed in the 'RFP ELIGIBILITY CRITERIA' section. Identify any requirements the company fails to meet, which should be considered potential deal-breakers.

### RFP ELIGIBILITY CRITERIA:
{rfp_chunk}

### COMPANY PROFILE:
{company_chunk}

**Task:**

1.  **Analyze:** Carefully examine each mandatory eligibility criterion described within the text under `### RFP ELIGIBILITY CRITERIA:`.
2.  **Compare:** For each mandatory requirement found, search the text under `### COMPANY PROFILE:` to find corresponding information (e.g., details on state registrations, certifications, past performance, financial status, etc.).
3.  **Identify Gaps:** Determine if the Company Profile provides evidence that *every single* mandatory requirement is met. Explicitly note any mandatory requirement listed in the RFP Criteria for which supporting evidence is missing or contradictory in the Company Profile. These are potential deal-breakers.
4.  **Decide:** Based *strictly* on the comparison between the two text sections, conclude if the company is Eligible or Not Eligible. If the Company Profile does not confirm that *all* mandatory RFP requirements are met, the verdict must be Not Eligible.
5.  **Justify:** Provide a clear explanation for the verdict. List the specific RFP requirements that the Company Profile *does* satisfy. Then, list the specific RFP requirements that the Company Profile *does not* satisfy (or does not provide information for), highlighting these as the reasons for ineligibility (deal-breakers).
6.  **Estimate Confidence:** Provide a confidence score (percentage) reflecting how certain you are of the verdict based *only* on the clarity and directness of the information match between the provided RFP criteria text and the Company Profile text.

**Required Output Format (Plain Text):**

Present your findings clearly in plain text using the following headings. Do **not** use JSON format.

first line should be the verdict, either "Eligible" or "Not Eligible".
✅ Eligible 
❌ Not Eligible
                                                  
**Confidence Sore:** 
Enter only similarity score here (0-100%)

**Met Requirements:**
* [List requirement 1 from RFP Criteria that is confirmed by Company Profile]
* [List requirement 2 from RFP Criteria that is confirmed by Company Profile]
    * ... (List all met requirements using bullet points)

**Unmet Requirements (Potential Deal-Breakers):**
(Include this section *only* if there are unmet requirements)
* **Requirement:** [Text of unmet requirement 1 from RFP Criteria]
    **Reason:** [Specific reason based on Company Profile, e.g., 'Company profile does not mention active registration in California.' or 'Company profile lacks information regarding CMMI certification level.']
* **Requirement:** [Text of unmet requirement 2 from RFP Criteria]
    **Reason:** [Specific reason based on Company Profile]
    * ... (List all unmet requirements and reasons)

**Justification Summary:**
[Provide a brief summary explaining the verdict, referencing the comparison, e.g., "Based on the provided profile, the company appears Eligible as the profile text addresses all listed mandatory criteria." or "Based on the provided profile, the company appears Not Eligible because the profile does not contain information to verify requirement X..."]

**Important:** Base your analysis *solely* on the text provided under 'RFP ELIGIBILITY CRITERIA' and 'COMPANY PROFILE'. Do not make assumptions or use external knowledge beyond interpreting these texts. If the Company Profile lacks sufficient detail to confirm a mandatory requirement, it counts as not met for this analysis.

Be specific and concise.
""")

formatted_prompt = eligibility_prompt.format(
    rfp_chunk=rfp_chunk,
    company_chunk=company_chunk
)
def get_eligibility_checker_chain():
    llm = ChatOpenAI(model="gpt-4o")  # You can use gpt-3.5-turbo if preferred
    prompt = eligibility_prompt
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain


In [53]:
eligibility_chain = get_eligibility_checker_chain()
response = eligibility_chain.run({
    "rfp_chunk": rfp_chunk,
    "company_chunk": company_chunk
})

print(response)

❌ Not Eligible

**Confidence Interval:** 
    75% (The information provided in both sections is somewhat clear, but there are significant omissions related to eligibility criteria.)

**Met Requirements:**
* Proposal must be signed in ink by an individual authorized to enter into contracts.
    * Verified by: "Signature: Meredith Chan (signed manually)."

**Unmet Requirements (Potential Deal-Breakers):**
* **Requirement:** Proposal must include a bank letter of creditworthiness.
    **Reason:** Company profile states "Bank Letter of Creditworthiness: Not Available."

* **Requirement:** Proposal received after the stated deadline shall be deemed late and will not be considered.
    **Reason:** No information is provided in the company profile regarding the proposal submission date, so compliance cannot be confirmed.

* **Requirement:** Evaluation of all Proposals will be based solely upon information contained in the proposal as a formal response.
    **Reason:** The company profile does

In [ ]:
def get_criteria_prompt():
    return PromptTemplate.from_template("""**Role:** You are an expert AI assistant specialized in analyzing Request for Proposals (RFPs) to identify mandatory requirements and evaluate company profile alignment.

    **Objective:** To extract all mandatory ('must-have') qualifications from the provided RFP text, categorize them, compare them against the Company Profile, and present the findings in a structured table and a brief summary.

    ### RFP TEXT:
    {rfp_chunk}

    ### COMPANY PROFILE:
    {company_chunk}

    **Instructions:**

    1.  **Identify & Extract Mandatory Criteria:** Carefully read the `### RFP TEXT:`. Identify and extract all explicit mandatory requirements. Pay close attention to keywords like "must," "shall," "required," "mandatory," "minimum," "prerequisite," etc.
    2.  **Categorize Requirements:** Group the extracted mandatory requirements into logical categories, primarily focusing on:
        * Required Experience (e.g., years, specific project types, domain knowledge)
        * Certifications (e.g., industry standards like ISO, CMMI; specific technology certifications)
        * Registrations (e.g., state/federal business registration, specific portals)
        * Other Mandatory Requirements (any other explicit 'must-have' items like security clearances, insurance minimums, specific tool proficiency, etc.)
    3.  **Analyze Company Profile:** For each identified mandatory requirement, thoroughly search and analyze the `### COMPANY PROFILE:` section to find relevant supporting information or evidence.
    4.  **Compare and Assess Alignment:** For *each* mandatory requirement, compare the requirement against the information found in the Company Profile. Determine the alignment status based *only* on the provided texts:
        * **Met:** The Company Profile clearly provides evidence that satisfies the requirement.
        * **Not Met:** The Company Profile clearly indicates the requirement is not satisfied, OR critically, lacks *any* relevant information pertaining to the requirement.
        * **Partially Met / Unclear:** The Company Profile provides some related information but does not fully or unambiguously satisfy the specific requirement as stated in the RFP.
    5.  **Generate Output:** Structure your findings precisely as requested below.

    **Required Output Format:**

    **1. Mandatory Requirements Analysis Table:**

    Present the findings in a Markdown table format with the following columns:

    | Mandatory Requirement (from RFP)                     | Company Profile Evidence / Notes                                     | Alignment Status              |
    | :--------------------------------------------------- | :------------------------------------------------------------------- | :---------------------------- |
    | [e.g., Must possess active ISO 9001 certification.] | [e.g., Profile states: "Certified ISO 9001:2015 since 2022."]        | Met                           |
    | [e.g., Minimum 5 years experience managing federal IT projects > $1M.] | [e.g., Profile: "Managed 3 federal projects ($2M, $5M, $1.5M) over 7 years."] | Met                           |
    | [e.g., Shall be registered in SAM.gov.]             | [e.g., Profile lists DUNS and CAGE code, but no explicit SAM mention.] | Unclear                       |
    | [e.g., Key personnel must hold Top Secret clearance.] | [e.g., Profile mentions "cleared personnel available" - no level specified.] | Partially Met / Unclear       |
    | [e.g., Must have experience with AWS cloud platform.]| [e.g., Profile details extensive Azure experience, no AWS mentioned.]   | Not Met                       |
    | ...                  | ...                                                  | ...                                                                  | ...                           |
    *(Ensure **all** identified mandatory requirements are listed row-by-row in this table)*
 
    Provide a brief (2-4 sentences) summary statement. This should concisely state the overall apparent alignment of the Company Profile with the RFP's mandatory requirements based *only* on the table above. Highlight any major strengths (e.g., "Strong alignment in experience") or critical gaps/areas of concern (e.g., "Key certifications mentioned in the profile are missing or unclear").

    **Important:**
    * Focus *exclusively* on criteria explicitly stated as mandatory in the `### RFP TEXT:`. Ignore desirable or 'nice-to-have' criteria unless stated as mandatory.
    * The 'Alignment Status' must be based *strictly* on the comparison between the RFP requirement and the text within the `### COMPANY PROFILE:`. If information is missing in the profile, mark it as 'Not Met' or 'Unclear', do not assume.""")

# 2. Initialize the LLMChain
def get_criteria_checker_chain():
    llm = ChatOpenAI(model="gpt-4o")  # Or gpt-3.5-turbo
    prompt = get_criteria_prompt()
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain


In [58]:
criteria_chain = get_criteria_checker_chain()
criteria_response = criteria_chain.run({
    "rfp_chunk": rfp_chunk,
    "company_chunk": company_chunk
})

print(criteria_response)

**1. Mandatory Requirements Analysis Table:**

| Category              | Mandatory Requirement (from RFP)                                     | Company Profile Evidence / Notes                                      | Alignment Status              |
| :-------------------- | :------------------------------------------------------------------- | :-------------------------------------------------------------------- | :---------------------------- |
| Other                 | All proposals must be signed in ink by a person authorized to enter into contractual agreements. | Profile states that Meredith Chan signed manually (no explicit connection to contractual authority). | Partially Met / Unclear       |
| Other                 | Proposal received after the stated deadline shall be deemed late and will not be considered. | No evidence or mention of timely submission is available in the profile. | Unclear                       |
| Other                 | Evaluation of all Proposals will be b

In [31]:
def get_checklist_prompt():
    return PromptTemplate.from_template("""
You are a government RFP submission assistant.

Given the RFP text, extract and structure all submission-related guidelines, such as:

- Page limits
- Font type/size
- Line spacing
- Margin requirements
- Table of Contents (TOC) instructions
- Submission method (email, portal)
- Submission format (PDF, Word, etc.)
- Attachments or forms that must be included
- Submission deadline (date/time)

### RFP TEXT:
{rfp_chunk}

Return a structured checklist with bullet points and section titles like:
📋 Submission Format:
📎 Attachments Required:
📅 Deadlines:
📬 Submission Method:
📝 Formatting Requirements:
""")

# 2. Create LLMChain
def get_checklist_generator_chain():
    llm = ChatOpenAI(model="gpt-4o")
    prompt = get_checklist_prompt()
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain

In [32]:
# Run the checklist generator
checklist_chain = get_checklist_generator_chain()
checklist_response = checklist_chain.run({
    "rfp_chunk": rfp_chunk
})

print(checklist_response)

Here's a structured checklist based on the provided RFP text:

📅 Deadlines:
- Submission deadline: Not specified in the provided text; refer to the link provided for the due date.

📬 Submission Method:
- Proposals must be submitted via the following link: https://www.mhmrtarrant.org/bids-and-rfps/

📋 Submission Format:
- All proposals must be signed in ink by an authorized person or office of the company.
- Unsigned proposals will be considered nonresponsive and will not be accepted.

📎 Attachments Required:
- Specific attachments or forms are not mentioned in the provided text. Refer to the full RFP for any required documents.

📝 Formatting Requirements:
- Font type/size, line spacing, and margin requirements are not specified in the provided text. Refer to the full RFP for any specific formatting guidelines.
- A Table of Contents (TOC) is not explicitly required in the provided text.

Please ensure to refer to the full RFP document for any additional details and requirements not cove

In [33]:

# 1. Prompt for Risk Analyzer
def get_risk_analyzer_prompt():
    return PromptTemplate.from_template("""
You are a legal contracts analyst specializing in government RFPs.

Analyze the following RFP clauses and identify any potentially risky or biased terms for the vendor, such as:
- Unilateral termination clauses
- Indemnity requirements
- Unlimited liability
- Payment terms
- Dispute resolution mechanisms

Assign a risk level (Low / Medium / High) for each risky clause, and suggest a redline or modification if needed.

### RFP CLAUSES:
{rfp_chunk}

Return a structured risk report like:

⚠️ Risk Report:

1. Clause: "The agency may terminate at any time without notice."
   - Risk Level: High
   - Suggestion: "Request mutual termination clause with 30-day notice."

2. Clause: "Vendor shall bear all liability for data loss."
   - Risk Level: Medium
   - Suggestion: "Negotiate shared responsibility with liability cap."

Include only relevant risk sections and omit safe clauses.
""")

# 2. Create Risk Analyzer Chain
def get_risk_analyzer_chain():
    llm = ChatOpenAI(model="gpt-4o")
    prompt = get_risk_analyzer_prompt()
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain


In [34]:
risk_chain = get_risk_analyzer_chain()
risk_response = risk_chain.run({
    "rfp_chunk": rfp_chunk
})

print(risk_response)

⚠️ Risk Report:

1. Clause: "The agency may terminate at any time without cause."
   - Risk Level: High
   - Suggestion: "Request mutual termination clause with a 30-day notice period for termination without cause to allow the vendor time to adjust operations."

2. Clause: "The vendor agrees to indemnify and hold harmless the agency from any claims, damages, or suits arising out of or resulting from the vendor's performance of services."
   - Risk Level: Medium
   - Suggestion: "Propose limiting indemnity to those costs arising from vendor's negligence or willful misconduct, and clarify that indemnity does not extend to indirect or consequential damages."

3. Clause: "Vendor shall assume unlimited liability for any damages arising from breach of contract or failure to perform."
   - Risk Level: High
   - Suggestion: "Negotiate a cap on liability to a reasonable amount, such as the total value of the contract or a defined multiple thereof, to protect against potentially devastating fina